# Joins

## Spark Set Up

In [1]:
## Imports
import re
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SparkSession

app_name = "week2_joins"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .config("spark.ui.port","42229")\
        .getOrCreate()
sc = spark.sparkContext

## Change the working directory
%cd /media

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/28 01:16:48 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/28 01:16:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/28 01:16:48 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/28 01:16:48 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


/media


### Joins

* join       
https://spark.apache.org/docs/latest/api/python/_modules/pyspark/rdd.html#RDD.join     
* leftOuterJoin   
https://spark.apache.org/docs/latest/api/python/_modules/pyspark/rdd.html#RDD.leftOuterJoin    
* rightOuterJoin   
https://spark.apache.org/docs/latest/api/python/_modules/pyspark/rdd.html#RDD.rightOuterJoin    
* fullOuterJoin   
https://spark.apache.org/docs/latest/api/python/_modules/pyspark/rdd.html#RDD.fullOuterJoin   
* cartesian   
https://spark.apache.org/docs/latest/api/python/_modules/pyspark/rdd.html#RDD.cartesian

In [2]:
## Let's try several joins and observe their behavior

## Inner Join
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("c", 8)])
x.join(y).collect()

[('a', (1, 2))]

In [3]:
## Left Join
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("c", 8)])
x.leftOuterJoin(y).collect()

[('a', (1, 2)), ('b', (4, None))]

In [4]:
## Right Join
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("c", 8)])
x.rightOuterJoin(y).collect()

[('a', (1, 2)), ('c', (None, 8))]

In [5]:
## Full Join
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("c", 8)])
x.fullOuterJoin(y).collect()

[('a', (1, 2)), ('b', (4, None)), ('c', (None, 8))]

In [6]:
## Cartesian Join
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("c", 8)])
x.cartesian(y).collect()

[(('a', 1), ('a', 2)),
 (('a', 1), ('c', 8)),
 (('b', 4), ('a', 2)),
 (('b', 4), ('c', 8))]

### Bigger Example

Let's use our frequency table from Alice. Let's check the results of the previous joins

In [7]:
## Let's create a bigger data
ALICE_TXT = 'file:///media' + "/data/alice.txt"
aliceRDD = sc.textFile(ALICE_TXT)

x_big = aliceRDD.flatMap(lambda line: re.findall('[a-z]+', line.lower())) \
                 .map(lambda word: (word, 1)) \
                 .reduceByKey(lambda a, b: a + b)\
                 .cache()
y = sc.parallelize([("a", 2), ("c", 8), ('s', 5), ('i', 10)])

In [26]:
%%time
## Let's run a join
x_big.join(y).collect()

CPU times: user 11.8 ms, sys: 8.14 ms, total: 19.9 ms
Wall time: 334 ms


[('s', (222, 5)), ('c', (5, 8)), ('i', (546, 10)), ('a', (695, 2))]

In [28]:
%%time
## Left Join
x_big.leftOuterJoin(y).take(10)

CPU times: user 16.6 ms, sys: 1.38 ms, total: 18 ms
Wall time: 226 ms


[('gutenberg', (98, None)),
 ('of', (638, None)),
 ('s', (222, 5)),
 ('anywhere', (3, None)),
 ('no', (100, None)),
 ('whatsoever', (2, None)),
 ('away', (28, None)),
 ('are', (77, None)),
 ('have', (87, None)),
 ('before', (42, None))]

In [29]:
%%time
## Right Join
x_big.rightOuterJoin(y).take(10)

CPU times: user 27.4 ms, sys: 1.16 ms, total: 28.5 ms
Wall time: 387 ms


[('s', (222, 5)), ('c', (5, 8)), ('i', (546, 10)), ('a', (695, 2))]

In [30]:
%%time
## Full Join
x_big.fullOuterJoin(y).take(10)

CPU times: user 19.9 ms, sys: 299 µs, total: 20.2 ms
Wall time: 197 ms


[('gutenberg', (98, None)),
 ('of', (638, None)),
 ('s', (222, 5)),
 ('anywhere', (3, None)),
 ('no', (100, None)),
 ('whatsoever', (2, None)),
 ('away', (28, None)),
 ('are', (77, None)),
 ('have', (87, None)),
 ('before', (42, None))]

In [32]:
%%time
## Cartesian .... Be Careful!! How many records will be produced?
x_big.count()

CPU times: user 7.52 ms, sys: 687 µs, total: 8.21 ms
Wall time: 83.6 ms


3006

In [33]:
%%time
## Cartesian .... 3006 * 4 = 12024 records!
print(x_big.cartesian(y).take(10))
print(x_big.cartesian(y).count())

[(('project', 88), ('a', 2)), (('gutenberg', 98), ('a', 2)), (('ebook', 13), ('a', 2)), (('of', 638), ('a', 2)), (('s', 222), ('a', 2)), (('adventures', 11), ('a', 2)), (('in', 435), ('a', 2)), (('wonderland', 7), ('a', 2)), (('this', 181), ('a', 2)), (('is', 131), ('a', 2))]
12024
CPU times: user 1.89 ms, sys: 13 ms, total: 14.9 ms
Wall time: 155 ms
